In [1]:
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt 
import numpy as np
import optuna
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import holidays 

from statsmodels.tsa.seasonal import STL


from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import category_encoders as ce
import xgboost as xgb
from xgboost import XGBRegressor
from ydata_profiling import ProfileReport
from sklearn.linear_model import LinearRegression

import catboost as cb
from scalecast.Forecaster import Forecaster

import lightgbm as lgb
import torch

import time

alt.data_transformers.disable_max_rows()
sklearn.set_config(transform_output="pandas")

## Import Data

In [3]:
train = pd.read_csv('train.csv')
blind = pd.read_csv('test.csv')

In [48]:

def is_holiday(row):
    """
    Determines if a given date is a holiday in the specified country.
    Optimized for Canada, Finland, and Italy.
    
    Parameters:
    row: pandas Series containing 'country' and 'date' columns
    
    Returns:
    bool: True if the date is a holiday in the given country, False otherwise
    """
    country = row['country']
    date = pd.to_datetime(row['date']).date()  # Convert to date only

    country_mapping = {
        'Canada': holidays.CA(),
        'Finland': holidays.FI(),
        'Italy': holidays.IT(), 
        'Kenya': holidays.KE(),
        'Norway': holidays.NO(),
        'Singapore': holidays.SG(),
    }
    
    try:
        # Get the holidays object for the country
        if country in country_mapping:
            country_holidays = country_mapping[country]
            return date in country_holidays
        else:
            return False
    except Exception as e:
        print(f"Error checking holiday for {country} on {date}: {e}")
        return False

# Create a cache of holiday objects to improve performance
def initialize_holiday_detection(df):
    """
    Initialize holiday detection by creating holiday objects for all unique years in the dataset.
    This improves performance by avoiding repeated creation of holiday objects.
    
    Parameters:
    df: pandas DataFrame containing 'date' column
    """
    years = pd.to_datetime(df['date']).dt.year.unique()
    holiday_cache = {
        'Canada': {year: holidays.CA(years=year) for year in years},
        'Finland': {year: holidays.FI(years=year) for year in years},
        'Italy': {year: holidays.IT(years=year) for year in years}, 
        'Kenya': {year: holidays.KE(years=year) for year in years},
        'Norway': {year: holidays.NO(years=year) for year in years},
        'Singapore': {year: holidays.SG(years=year) for year in years}

    }
    
    def is_holiday_cached(row):
        country = row['country']
        date = pd.to_datetime(row['date']).date()
        year = date.year
        
        try:
            if country in holiday_cache:
                return date in holiday_cache[country][year]
            return False
        except Exception as e:
            print(f"Error checking holiday for {country} on {date}: {e}")
            return False
    
    return is_holiday_cached

# Example usage:
# First initialize the cached version
holiday_checker = initialize_holiday_detection(train)

# Then apply it to create the holiday flag
train['is_holiday'] = train.apply(holiday_checker, axis=1)

In [ ]:
holiday_checker = initialize_holiday_detection(blind)
blind['is_holiday'] =blind.apply(holiday_checker, axis=1)
X_trial = feature_eng(blind)[1]


In [31]:
X_trial.head(2)

,is_holiday,day_of_year,month,year,country_Canada,country_Finland,country_Italy,country_Kenya,country_Norway,country_Singapore,store_Discount Stickers,store_Premium Sticker Mart,store_Stickers for Less,product_Holographic Goose,product_Kaggle,product_Kaggle Tiers,product_Kerneler,product_Kerneler Dark Mode
id,,,,,,,,,,,,,,,,,,
230130,True,1,1,2017,True,False,False,False,False,False,True,False,False,True,False,False,False,False
230131,True,1,1,2017,True,False,False,False,False,False,True,False,False,False,True,False,False,False


In [6]:
def feature_eng(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])

    # Extract date features
    
    df['day_of_year'] = df['date'].dt.dayofyear
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year

    # Encode categorical features
    # Drop missing values and irrelevant columns
    imputer = SimpleImputer(strategy="mean")  # Change to "mean", "most_frequent", or "constant"
    scaler = MinMaxScaler()
    df = df.set_index('id')
    df = df.drop(columns=['date'])  # Dropping columns that are not useful for modeling
    # Separate features and target
    try: 
        df["num_sold"] = imputer.fit_transform(df[["num_sold"]])
       # df["num_sold"] = scaler.fit_transform(df[["num_sold"]])
       


    except: pass
    
    try: 
        
        X = df.drop(columns=['num_sold'])
        
        X = pd.get_dummies(X)

    except:
        X=pd.get_dummies(df)
    try:
        y = df['num_sold']
    except:
        y = None
    # Scale features
    #X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

    return df, X,  y

# Feature engineering
df, X, y = feature_eng(train)

# Splitting the data
SPLIT = 0.85
split_index = int(SPLIT * len(X))

X_train = X[:split_index]
y_train = y[:split_index]
X_test = X[split_index:]
y_test = y[split_index:]

# Verify shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (195610, 17)
y_train shape: (195610,)
X_test shape: (34520, 17)
y_test shape: (34520,)


In [47]:
X_train

,day_of_year,month,year,country_Canada,country_Finland,country_Italy,country_Kenya,country_Norway,country_Singapore,store_Discount Stickers,store_Premium Sticker Mart,store_Stickers for Less,product_Holographic Goose,product_Kaggle,product_Kaggle Tiers,product_Kerneler,product_Kerneler Dark Mode
id,,,,,,,,,,,,,,,,,
83223,195,7,2012,False,False,False,False,True,False,True,False,False,False,False,False,True,False
44320,128,5,2011,False,False,True,False,False,False,False,True,False,True,False,False,False,False
44288,128,5,2011,True,False,False,False,False,False,False,False,True,False,False,False,True,False
203719,73,3,2016,False,False,False,True,False,False,True,False,False,False,False,False,False,True
109741,124,5,2013,False,False,True,False,False,False,True,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,236,8,2013,False,False,False,False,False,True,False,True,False,False,False,False,False,True
103694,57,2,2013,True,False,False,False,False,False,False,True,False,False,False,False,False,True
131932,5,1,2014,False,False,False,False,False,True,False,False,True,False,False,True,False,False


In [70]:
y_train

id
0         752.527382
1         973.000000
2         906.000000
3         423.000000
4         491.000000
             ...    
195605    150.000000
195606    874.000000
195607    717.000000
195608    453.000000
195609    438.000000
Name: num_sold, Length: 195610, dtype: float64

In [68]:
blind

id
195610     170.0
195611    1047.0
195612     981.0
195613     513.0
195614     492.0
           ...  
230125     466.0
230126    2907.0
230127    2299.0
230128    1242.0
230129    1622.0
Name: num_sold, Length: 34520, dtype: float64

In [71]:
model = LinearRegression()

# Train the model using training data
model.fit(X_train.values, y_train.values)

# Make predictions on the test data
y_pred = model.predict(X_test.values)

# Evaluate the model using common metrics
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
print(f"Mean Absolute Error: {mae:.4f}")
print(f"MAPE: {mape:.4f}")

print(f"Mean Squared Error: {mse:.4f}")
print(f"R-squared: {r2:.4f}")

# Optionally: You can get the coefficients and intercept of the linear regression model
print(f"Coefficients: {model.coef_}")
print(f"Intercept: {model.intercept_}")

Mean Absolute Error: 248.1475
MAPE: 3.0422
Mean Squared Error: 118420.2639
R-squared: 0.6478
Coefficients: [ 3.77165272e+01  8.10233613e-02 -3.42425231e+00 -6.81727755e+00
  7.79357301e+01 -9.81025856e+00 -2.14775054e+02 -6.50285148e+02
  7.08026730e+02  8.89080004e+01 -3.08593593e+02  2.22084217e+02
  8.65093759e+01 -4.56236057e+02  4.90390358e+02  2.69673676e+02
 -1.99475825e+02 -1.04352152e+02]
Intercept: 14490.864200642352


In [74]:
def objective(trial, X_train, X_test, y_train, y_test, total_trials):
    """
    Optuna objective function for XGBoost hyperparameter optimization with GPU support.
    
    Parameters:
    trial: Optuna trial object
    X_train, X_test, y_train, y_test: Training and test data
    total_trials: Total number of trials to run
    """
    trial_start = time.time()
    print(f"🔄 Starting Trial {trial.number + 1} out of {total_trials}")
    
    # Check GPU availability
    gpu_available = torch.cuda.is_available()
    device = "gpu" if gpu_available else "cpu"
    tree_method = "hist" if gpu_available else "auto"
    
    # Define hyperparameters with reasonable ranges
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 18),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 10.0, log=True),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 7),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "enable_categorical": True,
        "device": device,
        "tree_method": tree_method
    }
    
    try:
        # Convert data to device-appropriate format
        if device == "gpu":
            X_train_device = xgb.DMatrix(X_train, y_train)
            X_test_device = xgb.DMatrix(X_test)
        else:
            X_train_device = X_train
            X_test_device = X_test
        
        # Train the model with early stopping
        model = xgb.XGBRegressor(
            **param,
            early_stopping_rounds=50,
            eval_metric=['mae', 'mape']
        )
        
        model.fit(
            X_train_device, y_train,
            eval_set=[(X_test_device, y_test)],
            verbose=False
        )
        
        # Make predictions
        preds = model.predict(X_test_device)
        mape = mean_absolute_percentage_error(y_test, preds)
        
        # Calculate additional metrics
        mae = np.mean(np.abs(y_test - preds))
        rmse = np.sqrt(np.mean((y_test - preds) ** 2))
        
        # Log results
        trial.set_user_attr('mae', mae)
        trial.set_user_attr('rmse', rmse)
        trial.set_user_attr('n_estimators_used', model.best_iteration or param['n_estimators'])
        
        trial_time = time.time() - trial_start
        print(f"✅ Trial {trial.number + 1} completed in {trial_time:.1f}s")
        print(f"   MAPE: {mape:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")
        
        return mape
        
    except Exception as e:
        print(f"❌ Trial {trial.number + 1} failed: {str(e)}")
        raise optuna.exceptions.TrialPruned()

def run_optimization(X_train, X_test, y_train, y_test, n_trials=5):
    """
    Run the complete optimization process and return the best model.
    """
    study = optuna.create_study(direction="minimize")
    
    try:
        study.optimize(
            lambda trial: objective(trial, X_train, X_test, y_train, y_test, n_trials),
            n_trials=n_trials,
            show_progress_bar=True
        )
        
        # Get best parameters and create final model
        best_params = study.best_params
        best_params.update({
            "enable_categorical": True,
            "device": "gpu" if torch.cuda.is_available() else "cpu",
            "tree_method": "hist" if torch.cuda.is_available() else "auto"
        })
        
        best_model = xgb.XGBRegressor(**best_params)
        best_model.fit(X_train, y_train)
        
        print("\n🏆 Best Trial Results:")
        print(f"MAPE: {study.best_value:.4f}")
        print("Best Parameters:", best_params)
        
        return best_model, study
        
    except KeyboardInterrupt:
        print("\n⚠️ Optimization interrupted by user")
        return None, study


best_model, study = run_optimization(X_train, X_test, y_train, y_test, n_trials=2)

# Make predictions with best model
if best_model is not None:
    predictions = best_model.predict(X_test)
    final_mape = mean_absolute_percentage_error(y_test, predictions)
    print(f"Final MAPE: {final_mape:.4f}")

[I 2025-01-30 13:26:17,001] A new study created in memory with name: no-name-ab083a5c-0b07-4b12-ba43-df4ce3f5a67e


  0%|          | 0/15 [00:00<?, ?it/s]

🔄 Starting Trial 1 out of 15
✅ Trial 1 completed in 2.7s
   MAPE: 1.8820, MAE: 110.6834, RMSE: 147.2241
[I 2025-01-30 13:26:19,756] Trial 0 finished with value: 1.8819735792729888 and parameters: {'n_estimators': 76, 'learning_rate': 0.021529971531891543, 'max_depth': 15, 'subsample': 0.9551041287765066, 'colsample_bytree': 0.9948239312763494, 'reg_alpha': 6.937097660895436, 'reg_lambda': 0.3382793882593333, 'min_child_weight': 2, 'gamma': 2.248265238783105}. Best is trial 0 with value: 1.8819735792729888.
🔄 Starting Trial 2 out of 15
✅ Trial 2 completed in 3.6s
   MAPE: 0.6619, MAE: 75.7065, RMSE: 120.6409
[I 2025-01-30 13:26:23,392] Trial 1 finished with value: 0.6618970142189052 and parameters: {'n_estimators': 204, 'learning_rate': 0.04510335348442594, 'max_depth': 15, 'subsample': 0.8138140076129938, 'colsample_bytree': 0.5445631689210862, 'reg_alpha': 0.6993973423962253, 'reg_lambda': 5.868284764973139, 'min_child_weight': 5, 'gamma': 1.67479363594156}. Best is trial 1 with value

In [43]:
import time

def objective(trial, X_train, X_test, y_train, y_test, total_trials):
    """
    Optuna objective function for XGBoost hyperparameter optimization with GPU support.
    """
    trial_start = time.time()
    print(f"🔄 Starting Trial {trial.number + 1} out of {total_trials}")
    
    gpu_available = torch.cuda.is_available()
    device = "gpu" if gpu_available else "cpu"
    tree_method = "hist" if gpu_available else "auto"
    
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 18),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 10.0, log=True),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 7),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "enable_categorical": True,
        "device": device,
        "tree_method": tree_method
    }
    
    try:
        if device == "gpu":
            X_train_device = xgb.DMatrix(X_train, y_train)
            X_test_device = xgb.DMatrix(X_test)
        else:
            X_train_device = X_train
            X_test_device = X_test
        
        model = xgb.XGBRegressor(
            **param,
            early_stopping_rounds=50,
            eval_metric=['mae', 'mape']
        )
        
        model.fit(
            X_train_device, y_train,
            eval_set=[(X_test_device, y_test)],
            verbose=False
        )
        
        preds = model.predict(X_test_device)
        mape = mean_absolute_percentage_error(y_test, preds)
        mae = np.mean(np.abs(y_test - preds))
        rmse = np.sqrt(np.mean((y_test - preds) ** 2))
        
        trial.set_user_attr('mae', mae)
        trial.set_user_attr('rmse', rmse)
        trial.set_user_attr('n_estimators_used', model.best_iteration or param['n_estimators'])
        
        trial_time = time.time() - trial_start
        print(f"✅ Trial {trial.number + 1} completed in {trial_time:.1f}s")
        print(f"   MAPE: {mape:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")
        
        return mape
        
    except Exception as e:
        print(f"❌ Trial {trial.number + 1} failed: {str(e)}")
        raise optuna.exceptions.TrialPruned()

def run_optimization(X_train, X_test, y_train, y_test, n_trials=2):
    """
    Run the complete optimization process and return the best model.
    """
    study = optuna.create_study(direction="minimize")
    
    try:
        study.optimize(
            lambda trial: objective(trial, X_train, X_test, y_train, y_test, n_trials),
            n_trials=n_trials,
            show_progress_bar=True
        )
        
        best_params = study.best_params
        best_params.update({
            "enable_categorical": True,
            "device": "gpu" if torch.cuda.is_available() else "cpu",
            "tree_method": "hist" if torch.cuda.is_available() else "auto"
        })
        
        best_model = xgb.XGBRegressor(**best_params)
        best_model.fit(X_train, y_train)
        
        print("\n🏆 Best Trial Results:")
        print(f"MAPE: {study.best_value:.4f}")
        print("Best Parameters:", best_params)
        
        return best_model, study
        
    except KeyboardInterrupt:
        print("\n⚠️ Optimization interrupted by user")
        return None, study

def train_country_models(df, X_train, X_test, y_train, y_test, n_trials=2):
    """
    Train separate XGBoost models for each country using Optuna optimization.
    """
    start_time = time.time()
    countries = df['country'].unique()
    country_models = {}
    
    for country in countries:
        country_start_time = time.time()
        print(f"\n🌍 Training model for {country}")
        print("=" * 50)
        
        train_country_mask = df.loc[X_train.index]['country'] == country
        test_country_mask = df.loc[X_test.index]['country'] == country
        
        X_train_country = X_train[train_country_mask]
        X_test_country = X_test[test_country_mask]
        y_train_country = y_train[train_country_mask]
        y_test_country = y_test[test_country_mask]
        
        if len(X_train_country) < 10:
            print(f"⚠️ Insufficient data for {country}. Skipping...")
            continue
            
        try:
            best_model, study = run_optimization(
                X_train_country, 
                X_test_country,
                y_train_country, 
                y_test_country,
                n_trials=n_trials
            )
            
            if best_model is not None:
                predictions = best_model.predict(X_test_country)
                final_mape = mean_absolute_percentage_error(y_test_country, predictions)
                
                country_models[country] = {
                    'model': best_model,
                    'study': study,
                    'mape': final_mape,
                    'n_train_samples': len(X_train_country),
                    'n_test_samples': len(X_test_country),
                    'best_parameters': study.best_params,
                    'training_time': time.time() - country_start_time
                }
                
                print(f"\n📊 Results for {country}:")
                print(f"Training samples: {len(X_train_country)}")
                print(f"Test samples: {len(X_test_country)}")
                print(f"Final MAPE: {final_mape:.4f}")
                print(f"Training time: {country_models[country]['training_time']:.1f}s")
                
        except Exception as e:
            print(f"❌ Error training model for {country}: {str(e)}")
            continue
    
    total_time = time.time() - start_time
    print(f"\n⏱️ Total training time: {total_time:.1f}s")
    
    print("\n📑 Overall Summary:")
    print("=" * 50)
    summary_data = []
    for country, results in country_models.items():
        summary_data.append({
            'Country': country,
            'MAPE': results['mape'],
            'Training Samples': results['n_train_samples'],
            'Test Samples': results['n_test_samples'],
            'Training Time (s)': results['training_time']
        })
    
    summary_df = pd.DataFrame(summary_data)
    print(summary_df.sort_values('MAPE'))
    
    return country_models

def predict_with_country_models(X_new, country_models, country_column):
    """
    Make predictions using country-specific models on new, unlabeled data.
    
    Parameters:
    X_new: DataFrame containing features for prediction (without target)
    country_models: Dictionary of trained models from train_country_models()
    country_column: String or array containing country values for each row
    
    Returns:
    array: Predictions for each row
    """
    predictions = np.zeros(len(X_new))
    
    for country, model_dict in country_models.items():
        # Get indices for this country
        country_mask = country_column == country
        
        if country_mask.any():
            # Get predictions for this country
            country_data = X_new[country_mask]
            predictions[country_mask] = model_dict['model'].predict(country_data)
    
    return predictions

# Example usage:

# Train the models
country_models = train_country_models(
    df=train,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    n_trials=2
)

# Make predictions
blind = blind.set_index('id')
predictions = predict_with_country_models(
    X_trial, 
    country_models, 
    blind['country'])   

overall_mape = mean_absolute_percentage_error(y_test, predictions)
print(f"\nOverall MAPE across all countries: {overall_mape:.4f}")


[I 2025-01-30 14:30:04,589] A new study created in memory with name: no-name-905c4780-94a0-49bb-bef4-8fa886414aa3



🌍 Training model for Canada


  0%|          | 0/2 [00:00<?, ?it/s]

🔄 Starting Trial 1 out of 2
✅ Trial 1 completed in 0.6s
   MAPE: 0.1334, MAE: 85.3408, RMSE: 125.5341
[I 2025-01-30 14:30:05,202] Trial 0 finished with value: 0.1334128914791621 and parameters: {'n_estimators': 75, 'learning_rate': 0.1754723833166494, 'max_depth': 15, 'subsample': 0.7386530033976806, 'colsample_bytree': 0.7562659196521215, 'reg_alpha': 0.5179925053583921, 'reg_lambda': 3.48468740676131, 'min_child_weight': 5, 'gamma': 3.125192267709992}. Best is trial 0 with value: 0.1334128914791621.
🔄 Starting Trial 2 out of 2
✅ Trial 2 completed in 0.4s
   MAPE: 0.1347, MAE: 85.7745, RMSE: 124.1040
[I 2025-01-30 14:30:05,643] Trial 1 finished with value: 0.1346745405762311 and parameters: {'n_estimators': 331, 'learning_rate': 0.10182164898796246, 'max_depth': 8, 'subsample': 0.6041867173486737, 'colsample_bytree': 0.6782371800835456, 'reg_alpha': 0.33912639610300443, 'reg_lambda': 0.381941112561235, 'min_child_weight': 5, 'gamma': 2.693944395982916}. Best is trial 0 with value: 0.1

[I 2025-01-30 14:30:06,203] A new study created in memory with name: no-name-07f827a3-7d25-45f5-9400-ef56a7852730


...

[I 2025-01-30 14:30:15,016] A new study created in memory with name: no-name-32e35159-13b5-4892-a180-876916d674d2



🏆 Best Trial Results:
MAPE: 0.1129
Best Parameters: {'n_estimators': 222, 'learning_rate': 0.12834252768216198, 'max_depth': 10, 'subsample': 0.5013200426384863, 'colsample_bytree': 0.8063182886950859, 'reg_alpha': 0.11049080780123227, 'reg_lambda': 0.20656902456860146, 'min_child_weight': 6, 'gamma': 4.832469707928234, 'enable_categorical': True, 'device': 'cpu', 'tree_method': 'auto'}

📊 Results for Norway:
Training samples: 32595
Test samples: 5760
Final MAPE: 0.1253
Training time: 1.7s

🌍 Training model for Singapore


  0%|          | 0/2 [00:00<?, ?it/s]

🔄 Starting Trial 1 out of 2
✅ Trial 1 completed in 0.5s
   MAPE: 0.0848, MAE: 81.4543, RMSE: 123.8013
[I 2025-01-30 14:30:15,483] Trial 0 finished with value: 0.08480727434851877 and parameters: {'n_estimators': 444, 'learning_rate': 0.13924027236332626, 'max_depth': 8, 'subsample': 0.7260072174553076, 'colsample_bytree': 0.7736848445693796, 'reg_alpha': 0.11533736730321459, 'reg_lambda': 0.8266076865190772, 'min_child_weight': 2, 'gamma': 1.2985067031910458}. Best is trial 0 with value: 0.08480727434851877.
🔄 Starting Trial 2 out of 2
✅ Trial 2 completed in 0.3s
   MAPE: 0.0877, MAE: 82.0505, RMSE: 123.3095
[I 2025-01-30 14:30:15,763] Trial 1 finished with value: 0.08774127539701118 and parameters: {'n_estimators': 278, 'learning_rate': 0.12543650096692632, 'max_depth': 4, 'subsample': 0.9050755524814427, 'colsample_bytree': 0.9440390910864425, 'reg_alpha': 7.902343515073573, 'reg_lambda': 2.0502475343588187, 'min_child_weight': 7, 'gamma': 3.851877080156634}. Best is trial 0 with val

ValueError: feature_names mismatch: ['day_of_year', 'month', 'year', 'country_Canada', 'country_Finland', 'country_Italy', 'country_Kenya', 'country_Norway', 'country_Singapore', 'store_Discount Stickers', 'store_Premium Sticker Mart', 'store_Stickers for Less', 'product_Holographic Goose', 'product_Kaggle', 'product_Kaggle Tiers', 'product_Kerneler', 'product_Kerneler Dark Mode'] ['is_holiday', 'day_of_year', 'month', 'year', 'country_Canada', 'country_Finland', 'country_Italy', 'country_Kenya', 'country_Norway', 'country_Singapore', 'store_Discount Stickers', 'store_Premium Sticker Mart', 'store_Stickers for Less', 'product_Holographic Goose', 'product_Kaggle', 'product_Kaggle Tiers', 'product_Kerneler', 'product_Kerneler Dark Mode']
training data did not have the following fields: is_holiday

In [39]:
X_trial

,is_holiday,day_of_year,month,year,country_Canada,country_Finland,country_Italy,country_Kenya,country_Norway,country_Singapore,store_Discount Stickers,store_Premium Sticker Mart,store_Stickers for Less,product_Holographic Goose,product_Kaggle,product_Kaggle Tiers,product_Kerneler,product_Kerneler Dark Mode
id,,,,,,,,,,,,,,,,,,
230130,True,1,1,2017,True,False,False,False,False,False,True,False,False,True,False,False,False,False
230131,True,1,1,2017,True,False,False,False,False,False,True,False,False,False,True,False,False,False
230132,True,1,1,2017,True,False,False,False,False,False,True,False,False,False,False,True,False,False
230133,True,1,1,2017,True,False,False,False,False,False,True,False,False,False,False,False,True,False
230134,True,1,1,2017,True,False,False,False,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328675,False,365,12,2019,False,False,False,False,False,True,False,True,False,True,False,False,False,False
328676,False,365,12,2019,False,False,False,False,False,True,False,True,False,False,True,False,False,False
328677,False,365,12,2019,False,False,False,False,False,True,False,True,False,False,False,True,False,False


In [28]:
predictions = predict_with_country_models(X_trial, country_models, blind)
submission_df = pd.DataFrame({"predictions": predictions})
submission_df = pd.merge(blind['id'], 
         submission_df,
         left_index=True,
         right_index=True)
submission_file = submission_df.to_csv("submission.csv", index=False)


KeyError: "None of [Index([230130, 230131, 230132, 230133, 230134, 230135, 230136, 230137, 230138,\n       230139,\n       ...\n       328670, 328671, 328672, 328673, 328674, 328675, 328676, 328677, 328678,\n       328679],\n      dtype='int64', name='id', length=98550)] are in the [index]"

In [34]:
X_trial

,is_holiday,day_of_year,month,year,country_Canada,country_Finland,country_Italy,country_Kenya,country_Norway,country_Singapore,store_Discount Stickers,store_Premium Sticker Mart,store_Stickers for Less,product_Holographic Goose,product_Kaggle,product_Kaggle Tiers,product_Kerneler,product_Kerneler Dark Mode
id,,,,,,,,,,,,,,,,,,
230130,True,1,1,2017,True,False,False,False,False,False,True,False,False,True,False,False,False,False
230131,True,1,1,2017,True,False,False,False,False,False,True,False,False,False,True,False,False,False
230132,True,1,1,2017,True,False,False,False,False,False,True,False,False,False,False,True,False,False
230133,True,1,1,2017,True,False,False,False,False,False,True,False,False,False,False,False,True,False
230134,True,1,1,2017,True,False,False,False,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328675,False,365,12,2019,False,False,False,False,False,True,False,True,False,True,False,False,False,False
328676,False,365,12,2019,False,False,False,False,False,True,False,True,False,False,True,False,False,False
328677,False,365,12,2019,False,False,False,False,False,True,False,True,False,False,False,True,False,False


In [35]:
blind

,id,date,country,store,product,is_holiday
0,230130,2017-01-01,Canada,Discount Stickers,Holographic Goose,True
1,230131,2017-01-01,Canada,Discount Stickers,Kaggle,True
2,230132,2017-01-01,Canada,Discount Stickers,Kaggle Tiers,True
3,230133,2017-01-01,Canada,Discount Stickers,Kerneler,True
4,230134,2017-01-01,Canada,Discount Stickers,Kerneler Dark Mode,True
...,...,...,...,...,...,...
98545,328675,2019-12-31,Singapore,Premium Sticker Mart,Holographic Goose,False
98546,328676,2019-12-31,Singapore,Premium Sticker Mart,Kaggle,False
98547,328677,2019-12-31,Singapore,Premium Sticker Mart,Kaggle Tiers,False
98548,328678,2019-12-31,Singapore,Premium Sticker Mart,Kerneler,False


In [46]:

# Feature engineering function
def feature_eng(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    
    # Extract date features
    df['day_of_year'] = df['date'].dt.dayofyear
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year

    # Encode categorical features
    imputer = SimpleImputer(strategy="mean")
    scaler = MinMaxScaler()
    df = df.set_index('id')
    df = df.drop(columns=['date'])  # Dropping columns that are not useful for modeling
    
    try: 
        df["num_sold"] = imputer.fit_transform(df[["num_sold"]])
    except: pass
    
    try: 
        X = df.drop(columns=['num_sold'])
        X = pd.get_dummies(X)
    except:
        X = pd.get_dummies(df)
    try:
        y = df['num_sold']
    except:
        y = None

    return df, X, y

# Feature engineering
df, X, y = feature_eng(train)

# Split the training data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Prepare the validation data (blind dataset)
blind, X_val, y_val = feature_eng(blind)

# Verify shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape if y_val is not None else "None")

# Define the objective function for Optuna
def objective(trial, X_train, X_test, y_train, y_test, total_trials):
    trial_start = time.time()
    print(f"🔄 Starting Trial {trial.number + 1} out of {total_trials}")
    
    gpu_available = torch.cuda.is_available()
    device = "gpu" if gpu_available else "cpu"
    tree_method = "hist" if gpu_available else "auto"
    
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 18),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 10.0, log=True),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 7),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "enable_categorical": True,
        "device": device,
        "tree_method": tree_method
    }
    
    try:
        model = xgb.XGBRegressor(
            **param,
            early_stopping_rounds=50,
            eval_metric=['mae', 'mape']
        )
        
        model.fit(
            X_train, y_train,
            eval_set=[(X_test, y_test)],
            verbose=False
        )
        
        preds = model.predict(X_test)
        mape = mean_absolute_percentage_error(y_test, preds)
        mae = np.mean(np.abs(y_test - preds))
        rmse = np.sqrt(np.mean((y_test - preds) ** 2))
        
        trial.set_user_attr('mae', mae)
        trial.set_user_attr('rmse', rmse)
        trial.set_user_attr('n_estimators_used', model.best_iteration or param['n_estimators'])
        
        trial_time = time.time() - trial_start
        print(f"✅ Trial {trial.number + 1} completed in {trial_time:.1f}s")
        print(f"   MAPE: {mape:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")
        
        return mape
        
    except Exception as e:
        print(f"❌ Trial {trial.number + 1} failed: {str(e)}")
        raise optuna.exceptions.TrialPruned()

# Run the optimization
def run_optimization(X_train, X_test, y_train, y_test, n_trials=10):
    study = optuna.create_study(direction="minimize")
    
    try:
        study.optimize(
            lambda trial: objective(trial, X_train, X_test, y_train, y_test),
            n_trials=n_trials,
            show_progress_bar=True
        )
        
        best_params = study.best_params
        best_params.update({
            "enable_categorical": True,
            "device": "gpu" if torch.cuda.is_available() else "cpu",
            "tree_method": "hist" if torch.cuda.is_available() else "auto"
        })
        
        best_model = xgb.XGBRegressor(**best_params)
        best_model.fit(X_train, y_train)
        
        print("\n🏆 Best Trial Results:")
        print(f"MAPE: {study.best_value:.4f}")
        print("Best Parameters:", best_params)
        
        return best_model, study
        
    except KeyboardInterrupt:
        print("\n⚠️ Optimization interrupted by user")
        return None, study

# Train the model
best_model, study = run_optimization(X_train, X_test, y_train, y_test, n_trials=10)

# Make predictions on the validation set
if best_model is not None:
    val_predictions = best_model.predict(X_val)

[I 2025-01-30 14:40:30,076] A new study created in memory with name: no-name-caa43e81-9af4-4a3c-bbbb-04825a7fc247


X_train shape: (184104, 17)
y_train shape: (184104,)
X_test shape: (46026, 17)
y_test shape: (46026,)
X_val shape: (98550, 17)
y_val shape: None


  0%|          | 0/10 [00:00<?, ?it/s]

🔄 Starting Trial 1
✅ Trial 1 completed in 4.8s
   MAPE: 1.2123, MAE: 107.4546, RMSE: 159.9806
[I 2025-01-30 14:40:34,863] Trial 0 finished with value: 1.2122551286532233 and parameters: {'n_estimators': 243, 'learning_rate': 0.014490968813260837, 'max_depth': 11, 'subsample': 0.9388212254105379, 'colsample_bytree': 0.538050187657469, 'reg_alpha': 0.9994256319040513, 'reg_lambda': 0.20632841926768464, 'min_child_weight': 7, 'gamma': 2.3317450474093206}. Best is trial 0 with value: 1.2122551286532233.
🔄 Starting Trial 2
✅ Trial 2 completed in 14.6s
   MAPE: 0.3449, MAE: 57.1146, RMSE: 103.5934
[I 2025-01-30 14:40:49,526] Trial 1 finished with value: 0.344871280697394 and parameters: {'n_estimators': 440, 'learning_rate': 0.025343975998466568, 'max_depth': 15, 'subsample': 0.9292685013134528, 'colsample_bytree': 0.9562967153734246, 'reg_alpha': 0.5610948367775704, 'reg_lambda': 3.3008189611528453, 'min_child_weight': 2, 'gamma': 3.3117040484940845}. Best is trial 1 with value: 0.344871280